In [1]:
import numpy as np
import pandas as pd
import os
import csv
import time
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, TensorDataset

In [2]:
sequence = 10


def generate_sequence_dataset(x, sequence=sequence):
    x_list = []
    y_list = []
    x_len = len(x)
    for i in range(x_len - sequence-4):
        x_list.append(x[i:i + sequence])
        y_list.append(x[:, 4][i + sequence+4:i + sequence + 5][0])
    return np.array(x_list), np.array(y_list)

In [3]:
_dir = './data/'
file_list = os.listdir(_dir)
print(file_list)  # show list
x_train_dat = np.zeros((1, sequence, 6))
y_train_dat = np.zeros(1)

for fn in file_list:
    if fn[-3:] == 'csv':
        df = pd.read_csv("%s/%s" % (_dir, fn),
                         sep=",",
                         header=0,
                         index_col=False)
        df = df.drop([
            'timestamp', 'device_name', 'fcnt', 'dr', 'channel', 'rssi', 'snr',
            'channel'
        ],
                     axis=1)
        x_dat, y_dat = generate_sequence_dataset(df.values)
        x_train_dat = np.append(x_train_dat, x_dat, axis=0)
        y_train_dat = np.append(y_train_dat, y_dat, axis=0)
print("x train dataset :", x_train_dat.shape)
print("y train dataset :", y_train_dat.shape)

['220219_022520.csv', '220321_220753.csv', '220223_001915.csv', '220319_134104.csv', '220220_233025.csv', '220227_023931.csv', '220308_230906.csv']
x train dataset : (1090, 10, 6)
y train dataset : (1090,)


In [4]:
print('pytorch :', torch.__version__)
print('graphic name :', torch.cuda.get_device_name())

pytorch : 1.10.2+cu102
graphic name : NVIDIA GeForce RTX 2060


In [5]:
def generate_tensor_loader(x_data,
                           y_data,
                           batch_size,
                           shuffle=True,
                           drop_last=True):
    x_tensor = torch.tensor(x_data, dtype=torch.float32)
    y_tensor = torch.tensor(y_data, dtype=torch.float32).view(-1)

    return DataLoader(TensorDataset(x_tensor, y_tensor),
                      batch_size=batch_size,
                      shuffle=shuffle,
                      drop_last=drop_last)

In [6]:
trainloader = generate_tensor_loader(x_train_dat, y_train_dat, batch_size=len(x_train_dat), shuffle=True, drop_last=True)

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
class Conv1d_LSTM(nn.Module):
    def __init__(self, num_layers, hidden_unit, dropout=0.5):
        super(Conv1d_LSTM, self).__init__()
        self.hidden_unit = hidden_unit
        self.conv1d = nn.Conv1d(in_channels=6,
                                out_channels=12,
                                kernel_size=4,
                                stride=1,
                                padding=2,
                                padding_mode='replicate')
        self.avgpooling = nn.AvgPool1d(kernel_size=2, stride=1)
        self.lstm = nn.LSTM(input_size=12,
                            hidden_size=hidden_unit,
                            num_layers=num_layers,
                            bias=True,
                            dropout=dropout,
                            bidirectional=False,
                            batch_first=True)

        self.fc_layer1 = nn.Linear(self.hidden_unit, self.hidden_unit // 2)
        self.relu = nn.ReLU()
        self.fc_layer2 = nn.Linear(self.hidden_unit // 2, 1)

    def forward(self, x):
        x = x.transpose(1, 2)

        x = self.conv1d(x)
        x = self.relu(x)
        x = self.avgpooling(x)
        x = x.transpose(1, 2)

        self.lstm.flatten_parameters()
        _, (hidden, _) = self.lstm(x)
        x = hidden[-1]

        x = self.fc_layer1(x)
        x = self.relu(x)
        x = self.fc_layer2(x)

        return x

In [9]:
class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps
        
    def forward(self,yhat,y):
        loss = torch.sqrt(self.mse(yhat,y) + self.eps)
        return loss

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = Conv1d_LSTM(num_layers=2, hidden_unit=200).to(device)
loss_function = RMSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001,
                       weight_decay=0.000001)  # optimizer

In [ ]:
view_train_epoch = 500
loss_arr = []
start = time.time()  # 시작 시간 저장

for epoch in range(200000):
    epoch = epoch + 0 + 1
    for train_iter, (train_x, train_y_true) in enumerate(trainloader):
        model.train()  # Train mode
        model.zero_grad()  # model zero initialize
        optimizer.zero_grad()  # optimizer zero initialize
        
        train_x, train_y_true = train_x.to(device), train_y_true.to(device)  # device(gpu)
        train_y_true = train_y_true.view(-1, 1)
        train_y_pred = model.forward(train_x)  # forward

        loss = loss_function(train_y_pred, train_y_true)  # loss function
        loss.backward()  # backward
        optimizer.step()  # optimizer
        _, pred_index = torch.max(train_y_pred, 1)
    
        if epoch % view_train_epoch == 0:
            loss_arr.append(loss.item())
            print("[Train] ({}, {}) RMSE = {:.5f}, lr={:.6f}".format(epoch, train_iter, loss.item(), optimizer.param_groups[0]['lr']))

[Train] (500, 0) RMSE = 13.44460, lr=0.000100
[Train] (1000, 0) RMSE = 13.34995, lr=0.000100
[Train] (1500, 0) RMSE = 13.25035, lr=0.000100
[Train] (2000, 0) RMSE = 13.13044, lr=0.000100
[Train] (2500, 0) RMSE = 12.99770, lr=0.000100
[Train] (3000, 0) RMSE = 12.83579, lr=0.000100


In [ ]:
f = plt.figure(figsize=[6, 3])
f.set_facecolor("white")
plt.style.use(['default'])


plt.grid(True)
plt.plot(loss_arr, label="RMSE")
plt.ylabel('RMSE')
plt.xlabel('500 epoch')

plt.legend()
plt.show()

In [ ]:
import torch.onnx

In [ ]:
input_names = ['Time series data']
output_names = ['Output']

x = torch.zeros(1, 10, 6).to(device)
torch.onnx.export(model, x, './model/regression_mode.onnx', input_names=input_names, output_names=output_names)
torch.save({'epoch' : epoch,
            'model_state_dict' : model.state_dict(),
            'optimizer_state_dict' : optimizer.state_dict(),
            'loss' : loss
           }, './model/inference.pt')